In [3]:
from bs4 import BeautifulSoup
import requests
import time
import os
from pprint import pprint

def tournament_dates(start=2018, end=2019): 
    year_range = range(start, end)
    month_range = range(1, 13, 2)
    for year in year_range:
        for month in month_range:
            yield "{}{:02}".format(year, month)
    
def get_banzuke(date):
    url = "http://sumodb.sumogames.de/Banzuke.aspx?b={}".format(date)
    response = requests.get(url)
    time.sleep(3) # sleep a little to make sure we don't kill the site
    return response

def save_response(path, name, response):
    file_name = os.path.join(path, "{}.html".format(name))
    with open(file_name, 'w') as f:
        f.write(response.text)

def save_tournament_pages(path, start, end):        
    for date in tournament_dates(start, end):
        print("Getting tournament: {}".format(date))
        response = get_banzuke(date)
        save_response(path, date, response)
    
from pymongo import MongoClient
from pprint import pprint
client = MongoClient('mongo', 27017)
db = client.sumo

In [ ]:
import pickle
ids = pickle.load(open("data/pickles/ids.pkl", 'rb'))

# Get Banzuke Information

In [6]:
save_tournament_pages("data/html_raw", 1927, 2018)

Getting tournament: 192701
Getting tournament: 192703
Getting tournament: 192705
Getting tournament: 192707
Getting tournament: 192709
Getting tournament: 192711


KeyboardInterrupt: 

# Get Rikishi Information

In [5]:
def get_rikishi(rikishi_id):
    url = "http://sumodb.sumogames.de/Rikishi.aspx?r={}".format(rikishi_id)
    response = requests.get(url)
    time.sleep(3) # sleep a little to make sure we don't kill the site
    return response

def get_ids(collection):
    for c in collection:
        rids = map(lambda r: r['rikishi_id'], c['rikishi'])
        yield from rids
        
def save_rikishi_pages(path, collection): 
    items = collection.find(projection={'rikishi.rikishi_id': 1})
    for rikishi_id  in get_ids(items):
        name = os.path.join(path, "{}.html".format(rikishi_id))
        
        if os.path.isfile(name):
            continue

        print("Getting rikishi: {}".format(rikishi_id))
        response = get_rikishi(rikishi_id)
        save_response(path, rikishi_id, response)
        
save_rikishi_pages("data/html_raw/rikishi", db.banzuke)

# Get Rikishi - Banzuke Information

In [9]:
def generate_rikishi_banzuke_urls(collection):
    url = "http://sumodb.sumogames.de/Rikishi_basho.aspx?r={}&b={}"
    for c in collection:
        print(c)
        bid = c['_id']
        rids = map(lambda r: r['rikishi_id'], c['rikishi'])
        yield from map(lambda rid: (bid, rid, url.format(rid, bid)), rids)

def get_url(url):
    response = requests.get(url)
    time.sleep(3) # sleep a little to make sure we don't kill the site
    return response

def save_rikishi_banzuke_pages(path, collection):
    items = collection.find({'_id': 201701}, projection={'rikishi.rikishi_id': 1})
    for bid, rid, url in generate_rikishi_banzuke_urls(items):
        name = os.path.join(path, "{}-{}.html".format(bid, rid))

        if os.path.isfile(name):
            continue

        print("Getting: {}".format(url))
        
        response = get_url(url)
        save_response(path, "{}-{}".format(bid, rid), response)
        
save_rikishi_banzuke_pages("data/html_raw/rikishi_banzuke", db.banzuke)

{'_id': 201701,
 'rikishi': [{'rikishi_id': 1219},
             {'rikishi_id': 1111},
             {'rikishi_id': 1123},
             {'rikishi_id': 1235},
             {'rikishi_id': 6468},
             {'rikishi_id': 11927},
             {'rikishi_id': 1226},
             {'rikishi_id': 5944},
             {'rikishi_id': 12130},
             {'rikishi_id': 6480},
             {'rikishi_id': 6599},
             {'rikishi_id': 11728},
             {'rikishi_id': 12210},
             {'rikishi_id': 6614},
             {'rikishi_id': 2832},
             {'rikishi_id': 6463},
             {'rikishi_id': 6491},
             {'rikishi_id': 6473},
             {'rikishi_id': 12055},
             {'rikishi_id': 1284},
             {'rikishi_id': 5967},
             {'rikishi_id': 11785},
             {'rikishi_id': 9079},
             {'rikishi_id': 11784},
             {'rikishi_id': 11786},
             {'rikishi_id': 12239},
             {'rikishi_id': 6642},
             {'rikishi_id': 67